In [ ]:
# Clone YOLOv5 repository
!git clone https://github.com/ultralytics/yolov5
%cd yolov5

# Install dependencies
!pip install -r requirements.txt


Cloning into 'yolov5'...
remote: Enumerating objects: 16701, done.
remote: Counting objects: 100% (242/242), done.
remote: Compressing objects: 100% (177/177), done.
remote: Total 16701 (delta 120), reused 134 (delta 65), pack-reused 16459
Receiving objects: 100% (16701/16701), 15.39 MiB | 27.07 MiB/s, done.
Resolving deltas: 100% (11419/11419), done.
/content/yolov5/yolov5


In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="DDraxPISyV5uF1zHxMH3")
project = rf.workspace("bottle-detection-cm2jz").project("bottle-detection-53jut")
version = project.version(1)

# Download the dataset
dataset = version.download("yolov5")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 8.6 MB/s eta 0:00:00
  Attempting uninstall: chardet
    Found existing installation: chardet 5.2.0
    Uninstalling chardet-5.2.0:
      Successfully uninstalled chardet-5.2.0
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Bottle-Detection-1 in yolov5pytorch:: 100%|██████████| 1749/1749 [00:00<00:00, 6276.00it/s]


In [ ]:
!ls ../Bottle-Detection-1


ls: cannot access '../Bottle-Detection-1': No such file or directory


In [ ]:
yaml_content = """
train: ../Bottle-Detection-1/train/images
val: ../Bottle-Detection-1/valid/images

nc: 1  # number of classes
names: ['bottle']  # class names
"""

with open("bottle.yaml", "w") as f:
    f.write(yaml_content)

# Verify the content of the created YAML file
!cat bottle.yaml



train: ../Bottle-Detection-1/train/images
val: ../Bottle-Detection-1/valid/images

nc: 1  # number of classes
names: ['bottle']  # class names


In [ ]:
# Train the YOLOv5 model
!python train.py --img 640 --batch 16 --epochs 50 --data bottle.yaml --weights yolov5s.pt


2024-06-30 19:57:57.070403: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-30 19:57:57.070459: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-30 19:57:57.071845: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
train: weights=yolov5s.pt, cfg=, data=bottle.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=50, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=data/hyps, resume_evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, opt

In [ ]:
!ls runs/train/
!ls runs/train/exp/weights/

# Download the best weights file if it exists
from google.colab import files
files.download('runs/train/exp/weights/best.pt')


exp
best.pt  last.pt


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import torch
from google.colab.patches import cv2_imshow
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
import io
from PIL import Image
import cv2

model = torch.hub.load('ultralytics/yolov5', 'custom', path='runs/train/exp/weights/best.pt') 

def capture_image():
    js = Javascript('''
        async function captureImage() {
            const video = document.createElement('video');
            const stream = await navigator.mediaDevices.getUserMedia({video: true});
            video.srcObject = stream;
            await new Promise((resolve) => video.onloadedmetadata = resolve);
            const canvas = document.createElement('canvas');
            canvas.width = video.videoWidth;
            canvas.height = video.videoHeight;
            canvas.getContext('2d').drawImage(video, 0, 0);
            stream.getVideoTracks()[0].stop();
            return canvas.toDataURL('image/jpeg', 0.8);
        }
        captureImage().then(dataURI => {
            google.colab.kernel.invokeFunction('notebook.receiveImage', [dataURI], {});
        });
    ''')
    display(js)

# Register callback function to receive the image data
def receive_image(data_uri):
    return data_uri

output.register_callback('notebook.receiveImage', receive_image)

# Capture an image
capture_image()

# Provide time for the JavaScript to execute and return the image data
import time
time.sleep(5)

# Assume data_uri has been set by JavaScript
data_uri = output.eval_js('google.colab.kernel.invokeFunction("notebook.receiveImage", [], {});')

def data_uri_to_image(data_uri):
    if data_uri:
        image_data = b64decode(data_uri.split(',')[1])
        image = Image.open(io.BytesIO(image_data))
        return cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
    else:
        raise ValueError("Failed to capture image.")

# Convert the data URI to an image
image = data_uri_to_image(data_uri)

# Perform inference
results = model(image)

# Render results on the frame
results.render()

# Display the frame
cv2_imshow(results.imgs[0])


Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-6-30 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


<IPython.core.display.Javascript object>